## XBRL US API - Python example  
This notebook contains example Python code to use the XBRL US API (https://xbrl.us/home/use/xbrl-api/)    
  
**Made by:** [Ties de Kok](https://www.tiesdekok.com)

## Imports

In [1]:
import os, re, sys, json
import requests

In [2]:
import pandas as pd
import numpy as np

In [3]:
import getpass

## Api documentation

https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails

## Generate an access token
The access token will expire after 60 minutes.

**You have two options:**

**Option 1:** obtain your own credentials

1. Obtain credentials here: https://xbrl.us/home/use/xbrl-api-community/#provisioning

If you use this script on your own computer I recommend to use the JSON file as described in option 1a, if you are using Binder I recommend to use option 1b.

>**Option 1a:**
>2. Update 'login_cred.json' with your `client_id`, `client_secret`, and `username`  
>3. Set `CREDENTIAL_TYPE` to `LOCAL`
>4. Input your password when asked

>**Option 1b:**
>2. Set `CREDENTIAL_TYPE` to `CLOUD`
>3. Input your details when asked

----

**Option 2:** Use my temporary credentials for demo purposes only

1. Set `CREDENTIAL_TYPE` to `TEMP`
2. Input your email address when asked


In [4]:
CREDENTIAL_TYPE = 'TEMP'

In [5]:
if CREDENTIAL_TYPE == 'TEMP':
    user_email = input(prompt="Please type your email address here: ")
    access_token = requests.get('https://tdekok-xbrlapi.builtwithdark.com/gettoken?platform=aaa-{}'.format(user_email)).text.replace('"', "")
elif CREDENTIAL_TYPE in ['LOCAL', 'CLOUD']:
    endpoint = 'https://api.xbrl.us'
    endpoint_auth = endpoint + '/oauth2/token'
    
    if CREDENTIAL_TYPE == 'LOCAL':
        with open('login_cred.json', 'r') as f:
            login_cred = json.loads(f.read())
            client_id = login_cred['client_id']
            client_secret = login_cred['client_secret']
            username = login_cred['username']
    else:
        client_id = input(prompt='Please input your client id here:')
        client_secret = getpass.getpass(prompt = 'Please input your client secret here:')
        username = input(prompt='Please input your username here:')
        
    password = getpass.getpass(prompt = 'Password: ')
    
    body_auth = {'grant_type' : 'password', 
                'client_id' : client_id, 
                'client_secret' : client_secret, 
                'username' : username,
                'password' : password,
                'platform' : 'uw-ipynb'}
    res = requests.post(endpoint_auth, data=body_auth)
    auth_json = res.json()
    access_token = auth_json['access_token']
else:
    print('Invalid credential type! Use TEMP, LOCAL, or CLOUD. See the instructions above.')

Please type your email address here: your@email.com


## Make a query

### Define the fields you would like returned

In [6]:
fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'dts.id',
         'fact.id',
         'report.filing-date',
         'period.fiscal-year',
         'period.instant',
         'report.document-type',
         'concept.local-name',
         'dimensions.count',
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'fact.value',
         'unit',
         'fact.decimals',
         'dimension.namespace',
         'member.namespace',
          'fact.has-dimensions'
         ]

### Define the XBRL elements (tags) you'd like 

In [7]:
XBRL_Elements = [
    'Assets',
    'Liabilities',
    'Revenues',
    'SalesRevenueNet',
    'SalesRevenueGoodsNet',
    'TotalRevenuesAndOtherIncome',
    'IntangibleAssetsNetExcludingGoodwill',
     'FiniteLivedIntangibleAssetsNet',
     'IndefiniteLivedIntangibleAssetsExcludingGoodwill'
                ]

### Define the companies you'd like

In [8]:
companies_cik = ['0000789019', ## Microsoft (MSFT)
                 '0001018724', ## Amazon (AMZN)
                 '0001652044', ## Alphabet (GOOG)
                 '0000051143', ## IBM (IBM)
                ]

### Define the years you'd like

In [9]:
#years = ['2019'] ## Use commas between for multiple years, e.g., '2018','2019'
years = [str(2013 + i) for i in range(8)] ## Years 2013 to 2020

### Specify if you want dimensions, no dimension, or all values

In [10]:
has_dimensions = 'ALL'  ## TRUE for require dimensions, FALSE for no dimensions, ALL for all values

### Specify the report types that you want

In [11]:
report_types = ['10-K', '10-K/A']

## Execute query

In [12]:
search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'

In [13]:
params = {'concept.local-name': ','.join(XBRL_Elements),
     'period.fiscal-period': 'Y',
     'period.fiscal-year': ','.join(years),
     'unit': 'USD',
     'entity.cik': ','.join(companies_cik),
     'report.document-type': ','.join(report_types),
     }  
if has_dimensions == 'ALL':
    dimension_options = ['TRUE', 'FALSE']
else:
    dimension_options = [has_dimensions]
    
all_res_list = []
for dimensions_param in dimension_options:
    print('Getting the data for: "fact.has-dimensions" = {}'.format(dimensions_param))
    ### Every request will return a max of 2000 results. So we loop until all results are retrieved. 
    done_retrieving_all_results = False
    offset = 0
    while not done_retrieving_all_results:
        params['fact.has-dimensions'] = dimensions_param
        params['fields'] = ','.join(fields) + ',fact.offset({})'.format(offset) 
        res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        
        ## Interpret as JSON
        res_json = res.json()
            
        ## Get the results
        ### Retrieve the data list
        res_list = res_json['data']
        
        ### Add to the results
        all_res_list += res_list
        
        ## Pagination check
        paging_dict = res_json['paging']
        print('Number of records retrieved: ', paging_dict['count'])
        if paging_dict['count'] >= 2000:
            offset += paging_dict['count']
        else:
            done_retrieving_all_results = True
    
## Convert to a DataFrame
res_df = pd.DataFrame(all_res_list)

Getting the data for: "fact.has-dimensions" = TRUE
Number of records retrieved:  2000
Number of records retrieved:  407
Getting the data for: "fact.has-dimensions" = FALSE
Number of records retrieved:  273


## Show results

In [14]:
res_df.sample(10)

,entity.cik,entity.name,dts.id,fact.id,report.filing-date,period.fiscal-year,period.instant,report.document-type,concept.local-name,dimensions.count,dimension.local-name,member.local-name,fact.value,unit,fact.decimals,dimension.namespace,member.namespace,fact.has-dimensions
2611,0000051143,INTERNATIONAL BUSINESS MACHINES CORPORATION,390929,235661738,2020-02-25,2018,2019-01-01,10-K,FiniteLivedIntangibleAssetsNet,0,,,3087000000,USD,-6,,,False
1019,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,278881,168981843,2018-02-27,2015,None,10-K,Revenues,2,ProductOrServiceAxis,SystemsSalesMember,1203000000,USD,-6,http://fasb.org/us-gaap/2017-01-31,http://www.ibm.com/20171231,True
400,0001018724,AMAZON COM INC,178008,141441186,2017-02-10,2014,None,10-K,SalesRevenueNet,1,StatementGeographicalAxis,GB,8341000000,USD,-6,http://fasb.org/us-gaap/2016-01-31,http://xbrl.sec.gov/country/2016-01-31,True
809,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,180044,145031880,2017-02-28,2015,None,10-K,Revenues,2,ConsolidationItemsAxis,IntersegmentEliminationMember,-499000000,USD,-6,http://fasb.org/us-gaap/2016-01-31,http://fasb.org/us-gaap/2016-01-31,True
2023,0000051143,INTERNATIONAL BUSINESS MACHINES CORPORATION,390929,235659696,2020-02-25,2019,None,10-K,Revenues,2,StatementBusinessSegmentsAxis,GlobalFinancingMember,1120000000,USD,-6,http://fasb.org/us-gaap/2019-01-31,http://www.ibm.com/20191231,True
1059,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,120438,90078318,2015-02-24,2014,None,10-K,Revenues,2,ProductOrServiceAxis,ServicesMember,2712000000,USD,-6,http://fasb.org/us-gaap/2014-01-31,http://www.ibm.com/20141231,True
1693,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,278881,168979675,2018-02-27,2016,None,10-K,Revenues,2,StatementBusinessSegmentsAxis,GlobalFinancingMember,3494000000,USD,-6,http://fasb.org/us-gaap/2017-01-31,http://www.ibm.com/20171231,True
1321,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,120438,90078313,2015-02-24,2013,None,10-K,Revenues,2,StatementBusinessSegmentsAxis,GlobalBusinessServicesMember,109000000,USD,-6,http://fasb.org/us-gaap/2014-01-31,http://www.ibm.com/20141231,True
1001,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,180044,145031387,2017-02-28,2016,None,10-K,Revenues,2,ProductOrServiceAxis,StorageMember,2083000000,USD,-6,http://fasb.org/us-gaap/2016-01-31,http://www.ibm.com/20161231,True
2011,0000051143,INTERNATIONAL BUSINESS MACHINES CORPORATION,390929,235658117,2020-02-25,2019,None,10-K,Revenues,2,StatementBusinessSegmentsAxis,SystemsMember,1528000000,USD,-6,http://fasb.org/us-gaap/2019-01-31,http://www.ibm.com/20191231,True


## Show dimensions example (if exists)

In [15]:
res_df[res_df['dimensions.count'] > 1].sort_values('dts.id')[['entity.name', 'dts.id', 'period.fiscal-year','period.instant',
                                                              'report.document-type', 'concept.local-name','dimensions.count', 'fact.has-dimensions',
                                                             'dimension.local-name', 'member.local-name', 'fact.value', 'unit']].head(10)

,entity.name,dts.id,period.fiscal-year,period.instant,report.document-type,concept.local-name,dimensions.count,fact.has-dimensions,dimension.local-name,member.local-name,fact.value,unit
2045,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,ConsolidationItemsAxis,OperatingSegmentsMember,10213000000,USD
2326,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,StatementBusinessSegmentsAxis,OnlineServicesDivisionMember,3284000000,USD
2046,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,ConsolidationItemsAxis,OperatingSegmentsMember,18680000000,USD
2047,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,ConsolidationItemsAxis,OperatingSegmentsMember,24738000000,USD
2347,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,StatementBusinessSegmentsAxis,EntertainmentAndDevicesDivisionMember,10213000000,USD
2048,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,ConsolidationItemsAxis,OperatingSegmentsMember,20295000000,USD
2349,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,StatementBusinessSegmentsAxis,WindowsDivisionMember,18680000000,USD
2049,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,ConsolidationItemsAxis,OperatingSegmentsMember,3284000000,USD
2324,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,StatementBusinessSegmentsAxis,ServerAndToolsMember,20295000000,USD
2351,MICROSOFT CORPORATION,73772,2013,None,10-K,SalesRevenueNet,2,True,StatementBusinessSegmentsAxis,MicrosoftBusinessDivisionMember,24738000000,USD
